# Production-Grade Customer Churn Modeling Pipeline
**Dataset**: `customers.csv`

This notebook implements a robust, production-ready machine learning pipeline for customer churn prediction. It incorporates best practices including:
- **Data Leakage Prevention**: Removing features recorded *after* the churn event.
- **Proper Evaluation**: Focusing on ROC-AUC, PR-AUC, and Recall for imbalanced data.
- **Robust Pipelines**: Consolidating preprocessing and modeling into a single `sklearn` Pipeline to prevent data leakage during cross-validation.
- **Hyperparameter Tuning**: Using `RandomizedSearchCV` for efficient optimization.
- **Explainability**: Using SHAP (SHapley Additive exPlanations) for transparent model interpretation.
- **Business Strategy**: Actionable retention insights based on model outputs.


In [ ]:
!pip install -q scikit-learn pandas numpy matplotlib seaborn xgboost shap


## 1. Imports & Configuration


In [ ]:
import warnings
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    roc_auc_score, average_precision_score, recall_score,
    precision_score, f1_score, accuracy_score,
    confusion_matrix, ConfusionMatrixDisplay,
    roc_curve, precision_recall_curve, auc
)

import shap
from xgboost import XGBClassifier

warnings.filterwarnings('ignore')
sns.set_theme(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)


## 2. Data Loading & Leakage Audit
**Data Leakage** occurs when a model is trained using information that won't be available at prediction time. 
For churn, features like `last_active`, `days_since_last_login`, or `month_last_active` are highly leaky because a customer who churned naturally stops logging in. Using these allows the model to "cheat" by detecting the *result* of churn rather than *predicting* it in advance.

We will proactively drop these leaky features.


In [ ]:
DATA_PATH = Path("../data/customers.csv")
df = pd.read_csv(DATA_PATH)

# Target Variable
TARGET = "churned"

print(f"Original shape: {df.shape}")

# ----- LEAKAGE AUDIT & REMOVAL -----
# Features directly caused by the churn event
leaky_features = ["last_active", "days_since_last_login", "month_last_active"]

# IDs and dates that are not predictive or have been parsed
# We will extract the month/year from signup_date, then drop it.
df["signup_date"] = pd.to_datetime(df["signup_date"], errors="coerce")
df["signup_month"] = df["signup_date"].dt.month
df["signup_year"] = df["signup_date"].dt.year

drop_cols = leaky_features + ["customer_id", "signup_date"]
existing_drop_cols = [c for c in drop_cols if c in df.columns]

df = df.drop(columns=existing_drop_cols)
print(f"Shape after removing leaky features/IDs: {df.shape}")
df.head(3)


## 3. Train-Test Split
We use a **Stratified Split** to ensure both the training and testing sets maintain the underlying 77/23 class ratio. Splitting *before* any preprocessing guarantees no data leakage (e.g., target statistics bleeding into the training data through scalers or imputers).


In [ ]:
X = df.drop(columns=[TARGET])
y = df[TARGET]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=RANDOM_STATE, 
    stratify=y  # Essential for imbalanced datasets
)

neg, pos = (y_train == 0).sum(), (y_train == 1).sum()
scale_pos_weight = neg / pos

print(f"Training Set: {X_train.shape[0]} rows (Churn Ratio: {y_train.mean():.2%})")
print(f"Testing Set:  {X_test.shape[0]} rows (Churn Ratio: {y_test.mean():.2%})")
print(f"Class Imbalance Ratio (Neg/Pos): {scale_pos_weight:.2f}")


## 4. Preprocessing Pipeline
We build a robust `ColumnTransformer` embedded inside an `sklearn.pipeline.Pipeline`. 
- **Numeric Features**: Median imputation (robust to outliers) + Standard Scaling.
- **Categorical Features**: Constant imputation + One-Hot Encoding (preventing dummy trap issues natively inside tree models, while `handle_unknown='ignore'` guards against unseen test categories).


In [ ]:
numeric_features = X_train.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = X_train.select_dtypes(include=["object", "string", "category"]).columns.tolist()

numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="constant", fill_value="missing")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features)
    ]
)


## 5. Model Training & Hyperparameter Tuning
We use **XGBoost**, which natively handles tabular capabilities exceptionally well.
- **`scale_pos_weight`**: Accounts for the 3.3:1 imbalance natively, optimizing the loss function to penalize missed churners heavily.
- **`RandomizedSearchCV`**: Tunes max_depth, learning rate, and subsampling efficiently through 3-fold cross-validation.


In [ ]:
# 1. Define the base pipeline
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", XGBClassifier(
        scale_pos_weight=scale_pos_weight, # Handles class imbalance
        eval_metric="logloss",
        random_state=RANDOM_STATE,
        n_jobs=-1
    ))
])

# 2. Define Hyperparameter space
param_grid = {
    "classifier__n_estimators": [100, 200, 300],
    "classifier__max_depth": [3, 5, 7],
    "classifier__learning_rate": [0.01, 0.05, 0.1],
    "classifier__subsample": [0.8, 1.0],
    "classifier__colsample_bytree": [0.8, 1.0]
}

# 3. Stratified K-Fold for robust CV
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=RANDOM_STATE)

# 4. RandomizedSearchCV optimizing for PR-AUC (best for imbalanced classification)
grid_search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_grid,
    n_iter=10,             # Keep low for notebook execution speed
    scoring="average_precision", # PR-AUC Score 
    cv=cv,
    verbose=1,
    random_state=RANDOM_STATE,
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

print(f"Best parameters: {grid_search.best_params_}")
best_model = grid_search.best_estimator_


## 6. Model Evaluation
For imbalanced datasets, **Accuracy is a misleading metric**. Instead, we focus on:
- **ROC-AUC**: Ability to rank churners higher than non-churners.
- **PR-AUC (Average Precision)**: Quality of the positive (churn) predictions.
- **Recall**: The percentage of actual churners we successfully identified.


In [ ]:
# Predictions
y_pred_proba = best_model.predict_proba(X_test)[:, 1]

# Default threshold 0.5 predictions
y_pred = best_model.predict(X_test)

metrics = {
    "ROC-AUC": roc_auc_score(y_test, y_pred_proba),
    "PR-AUC (Avg Precision)": average_precision_score(y_test, y_pred_proba),
    "Recall": recall_score(y_test, y_pred),
    "Precision": precision_score(y_test, y_pred),
    "F1-Score": f1_score(y_test, y_pred),
    "Accuracy": accuracy_score(y_test, y_pred)
}

print("--- Test Set Performance (Threshold=0.5) ---")
for k, v in metrics.items():
    print(f"{k:<25}: {v:.4f}")

# Plot Confusion Matrix
fig, ax = plt.subplots(figsize=(6, 5))
ConfusionMatrixDisplay.from_predictions(
    y_test, y_pred, 
    display_labels=["Retained", "Churned"], 
    cmap="Blues", ax=ax, colorbar=False
)
ax.set_title("Confusion Matrix (Test Set)")
ax.grid(False)
plt.show()


### 6.1 Performance Curves (ROC & Precision-Recall)


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# ROC Curve
fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
axes[0].plot(fpr, tpr, color="#2ecc71", lw=2, label=f"AUC = {metrics['ROC-AUC']:.3f}")
axes[0].plot([0, 1], [0, 1], color="gray", linestyle="--")
axes[0].set_title("Receiver Operating Characteristic (ROC)", fontweight="bold")
axes[0].set_xlabel("False Positive Rate")
axes[0].set_ylabel("True Positive Rate")
axes[0].legend(loc="lower right")

# PR Curve
prec, rec, _ = precision_recall_curve(y_test, y_pred_proba)
axes[1].plot(rec, prec, color="#e74c3c", lw=2, label=f"PR-AUC = {metrics['PR-AUC (Avg Precision)']:.3f}")
# Baseline PR is the proportion of positives
baseline = y_test.mean()
axes[1].axhline(baseline, color="gray", linestyle="--", label=f"Baseline ({baseline:.3f})")
axes[1].set_title("Precision-Recall Curve", fontweight="bold")
axes[1].set_xlabel("Recall")
axes[1].set_ylabel("Precision")
axes[1].legend(loc="upper right")

plt.tight_layout()
plt.show()


## 7. Decision Threshold Tuning
The default classification threshold evaluates at `0.5`. However, in business scenarios where the cost of a missed churner (False Negative) is high, we can lower the threshold to increase Recall at the expense of Precision.


In [ ]:
thresholds = np.linspace(0.1, 0.9, 50)
precisions, recalls, f1_scores = [], [], []

for t in thresholds:
    y_hat = (y_pred_proba >= t).astype(int)
    precisions.append(precision_score(y_test, y_hat, zero_division=0))
    recalls.append(recall_score(y_test, y_hat))
    f1_scores.append(f1_score(y_test, y_hat))

plt.figure(figsize=(10, 5))
plt.plot(thresholds, precisions, label="Precision", color="#3498db", lw=2)
plt.plot(thresholds, recalls, label="Recall", color="#e74c3c", lw=2)
plt.plot(thresholds, f1_scores, label="F1 Score", color="#f39c12", lw=2)

best_f1_idx = np.argmax(f1_scores)
best_thresh = thresholds[best_f1_idx]

plt.axvline(best_thresh, color="gray", linestyle="--", label=f"Max F1 Threshold ({best_thresh:.2f})")
plt.title("Metrics Across Decision Thresholds", fontweight="bold")
plt.xlabel("Probability Threshold")
plt.ylabel("Score")
plt.legend()
plt.show()

print(f"To maximize F1-Score, set the prediction threshold to: {best_thresh:.2f}")


## 8. Model Explainability (SHAP & Feature Importance)
Black-box models require interpretability for business trust. We extract the generated feature names from the `ColumnTransformer` and use `Feature Importances` and `SHAP` values to explain what drives churn.


In [ ]:
# 1. Extract mapped feature names from Pipeline
ohe_features = best_model.named_steps["preprocessor"].named_transformers_["cat"].named_steps["onehot"].get_feature_names_out(categorical_features)
all_feature_names = numeric_features + list(ohe_features)

# 2. Plot Global Feature Importances
xgb_model = best_model.named_steps["classifier"]
importances = xgb_model.feature_importances_

imp_df = pd.DataFrame({"Feature": all_feature_names, "Importance": importances})
imp_df = imp_df.sort_values(by="Importance", ascending=False).head(15)

plt.figure(figsize=(10, 6))
sns.barplot(data=imp_df, x="Importance", y="Feature", palette="viridis")
plt.title("Top 15 Most Important Features (XGBoost Gain)", fontweight="bold")
plt.show()


In [ ]:
# 3. SHAP Summary Plot
# We must pass the preprocessed data to the SHAP explainer
X_train_preprocessed = best_model.named_steps["preprocessor"].transform(X_train)
X_test_preprocessed = best_model.named_steps["preprocessor"].transform(X_test)

# Optional: Convert to DataFrame for pretty feature names in SHAP plot
X_test_sh = pd.DataFrame(X_test_preprocessed, columns=all_feature_names)

# Initialize explainer
explainer = shap.TreeExplainer(xgb_model)
shap_values = explainer.shap_values(X_test_sh)

plt.title("SHAP Feature Impacts on Churn Prediction")
shap.summary_plot(shap_values, X_test_sh, plot_type="dot")


## 9. Business Interpretation & Retention Strategy

Based on the pipeline evaluation and SHAP explanations, the machine learning model reveals the following actionable business insights:

### Interpretation of Results
1. **The Leakage Audit Restored Validity**: By removing metrics like `days_since_last_login` and `last_active`, the model now predicts churn *proactively* based on engagement and behavioral signals rather than simply recognizing inactive accounts.
2. **Class Imbalance Addressed**: The `scale_pos_weight` parameter forced the model to prioritize churners. Combined with **threshold tuning**, the business can now actively choose to flag more at-risk users by lowering the threshold (e.g., to ~0.35) to boost the Recall rate.
3. **Key Drivers of Churn**: 
    - *(Referencing the SHAP summary plot)*: The visualizations will show exactly which factors push a customer's probability assigned near 1 (Churn) vs 0 (Retained). E.g., low `avg_session_minutes`, a high amount of `num_tickets_90d`, or specific `plan` tiers.

### Recommended Retention Strategy
* How to use this model in production:
1. **Weekly Scoring Batch**: Run the active customer base through this pipeline every week to generate a "Churn Risk Score" (`predict_proba`).
2. **Targeted Interventions**:
    - **High Risk (Top 10% Probability)**: Trigger hands-on Customer Success outreach or aggressive discounts.
    - **Medium Risk**: Enroll in automated re-engagement email drip campaigns highlighting features they haven't used recently.
3. **Product Feedback Loop**: If the SHAP analysis shows `payment_failures_180d` is a massive driver of churn, the business should invest engineering resources into smoother dunning and payment retry processes. If support wait times (`num_tickets_90d`) drive churn, escalate support SLA protocols.

